In [ ]:
import osmium
import osmium.filter

import geopandas as gpd
import shapely.geometry as sgeo

In [ ]:
if "snakemake" in locals():
    input_path = snakemake.input["osm"]
    areas_path = snakemake.input["areas"]
    output_path = snakemake.output[0]
    area = snakemake.params["area"]

else:
    input_path = "../../resources/paris/osm/ile-de-france-latest.osm.pbf"
    areas_path = "../../resources/paris/areas.gpkg"
    output_path = "../../results/paris/networks/network.osm"
    area = "network"

In [6]:
# Load area
df_areas = gpd.read_file(areas_path)
df_areas = df_areas[df_areas["name"] == area]
assert len(df_areas) == 1
area = df_areas.to_crs("EPSG:4326")["geometry"].values[0]

In [7]:
# Filter the data set for highways and by shape
with osmium.BackReferenceWriter(output_path, ref_src = input_path, overwrite = True) as  writer:
    for item in osmium.FileProcessor(input_path).with_filter(osmium.filter.KeyFilter("highway")).with_locations().with_filter(osmium.filter.GeoInterfaceFilter()):
        geometry = sgeo.shape(item.__geo_interface__["geometry"])

        if area.intersects(geometry):
            writer.add(item)